In [114]:
import numpy as np
import pandas as pd

# importing the requests library
import requests

In [115]:
## Resources ##
# [1] https://www.arcgis.com/home/item.html?id=124c2187da8c41c59bde04fa67eb2872
# [2] https://data-wake.opendata.arcgis.com/datasets/Wake::restaurants-in-wake-county/api

In [163]:
## Sourcing Data ##

# Source restaurants data in wake county from wakegov

# keep pagination in mind 
FIRST_1000 = 'https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?where=1%3D1&outFields=*&returnGeometry=false&outSR=4326&f=json'
SECOND_1000 = 'https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?where=1%3D1&outFields=*&returnGeometry=false&outSR=4326&f=json&resultOffset=1000&resultRecordCount=1000'
THIRD_1000 = 'https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?where=1%3D1&outFields=*&returnGeometry=false&outSR=4326&f=json&resultOffset=2000&resultRecordCount=1000'
FINAL_640 = 'https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?where=1%3D1&outFields=*&returnGeometry=false&outSR=4326&f=json&resultOffset=3000&resultRecordCount=1000'

rows = []
# Sending get request and saving the response as response object
for val in [FIRST_1000, SECOND_1000, THIRD_1000, FINAL_640]:
    r = requests.get(url = val)
    data = r.json()['features']
    for d in data:
        rows.append(d['attributes'])

# extracting data in json 
df = pd.DataFrame(rows)

# restaurants only
df = df.loc[df['FACILITYTYPE'] == 'Restaurant']

In [165]:
# validate 
print(df.shape)
display(df.head())

(2381, 15)


,OBJECTID,HSISID,NAME,ADDRESS1,ADDRESS2,CITY,STATE,POSTALCODE,PHONENUMBER,RESTAURANTOPENDATE,FACILITYTYPE,PERMITID,X,Y,GEOCODESTATUS
2,1838439,04092017585,MCALISTER'S DELI #100958,1061 PINE PLAZA DR,None,APEX,NC,27523,(919) 362-6518,1493251200000,Restaurant,19935,-78.829393,35.746571,M
5,1838442,04092018454,Deja Brew,5001 FALLS OF NEUSE RD,None,RALEIGH,NC,27609,(919) 239-4612,1601510400000,Restaurant,19953,-78.615575,35.849783,M
6,1838443,04092016734,GROCERY BOY JR.,1912 W GARNER RD,None,GARNER,NC,27529,(919) 772-5018,1408665600000,Restaurant,19956,-78.625807,35.725984,M
7,1838444,04092018583,Wendys,2007 NW CARY PKY,None,MORRISVILLE,NC,27560-5908,(919) 465-1979,1616544000000,Restaurant,19958,-78.812350,35.805386,M
8,1838445,04092015676,ROADRUNNERS PIZZA,8420-120 LOUISBURG RD,None,RALEIGH,NC,27616,(919) 266-7606,1279843200000,Restaurant,19951,-78.510921,35.894136,M
